Scientific Computing Coursework EMAT30008

In [7]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from ODEsolver import solve_odes, error_difference, func1, true_euler_number

ImportError: cannot import name 'error_difference' from 'ODEsolver' (c:\Users\smith\OneDrive\Documents\Eng Maths Yr 3\Scientific Computing\EMAT30008\ODEsolver.py)

Ordinary Differential Equation solver

The purpose of this software is to solve an ordinary differential equation (ODE) or system of ODEs using either the Euler method or the fourth order Runge-Kutta (RK4) method. The script uses three functions to do this, euler_step, RK4_step and solve_odes. The euler_step and RK4_step have the same function, they recieve an ODE as an input coupled with an initial x and time value and return an updated x and time value. The solve_odes function takes the ODE system as an input, along with a starting value of x, a starting time value, a maximum step size and a solver option which is set to RK4 as a default. The solve_odes funciton, calculates the number of steps, and iterates over the steps, implementing either RK4 or euler at each time step and adding the solutions to an array of numerical solutions to the ODE system. The solve_odes also receives an any number of keyword arguments with the use of the optinal keyword arguments **kwargs. I decided to use **kwargs instead of *args as it is convenient to be able to explicitly define the optional inputs to functions. 

I have also created an errors function which compares the accuracy of the euler and RK4 methods applied to the ODE x. = x (which is the definition of the mathematical constant Eulers number.) I am therefore evaluating the accuracy using the absolute difference between the solution at increasing time step and the true value (np.exp). 

In [4]:
pars = 1
error_difference(func1, x0=1, t0=0, t1=1, true_function = true_euler_number, pars = pars)

TypeError: object of type 'int' has no len()